In [264]:
#imports 
import pandas as pd
import numpy as np
import os
import pickle as pkl
import datetime as datetime
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as sm
import dgl.function as fn
from tqdm import tqdm
import networkx as nx
from src.utils.TradeNetwork import TradeNetwork

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from src.pygcn import SiameseGNN

#imports for graph creation
import torch
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

#imports for graph learning
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
from tqdm import trange
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms

## All Events

In [265]:
years = range(1962,2019)

train_years = [2005, 1969, 2002, 1997, 1993, 1982, 2001, 2000, 1962, 1985, 1978, 2016, 1986, 1987, 1989, 1971, 2013, 1996, 1995, 1967, 2017, 1974, 1990, 1977, 1980, 2014, 1965, 1984, 2006, 1973, 1968, 1981, 1970, 1991]
val_years = [1975, 1983, 2009, 1966, 1999, 1988, 2007, 1979, 1972, 2015, 2003]
test_years = [1963, 1964, 1976, 1992, 1994, 1998, 2004, 2008, 2010, 2011, 2012, 2018]

In [274]:
all_nodes = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD',
       'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA',
       'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN',
       'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA',
       'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FSM',
       'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI',
       'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR',
       'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'LTU',
       'LUX', 'LVA', 'MAC', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL',
       'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT',
       'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR',
       'NPL', 'NRU', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW',
       'PNG', 'POL', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', 'RUS',
       'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR',
       'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM',
       'SYC', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON',
       'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA',
       'UZB', 'VCT', 'VEN', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB',
       'ZWE']

In [266]:
#12 features
import pickle as pkl
with open("src/pygcn/train_graphs.pickle", "rb") as f:
    train_graphs = pkl.load(f)

with open("src/pygcn/val_graphs.pickle", "rb") as f:  
    val_graphs = pkl.load(f)

with open("src/pygcn/test_graphs.pickle", "rb") as f:         
    test_graphs = pkl.load(f)

In [268]:
from torch_geometric.data import DataLoader
test_loader = DataLoader(test_graphs, batch_size=4)
train_loader = DataLoader(train_graphs, batch_size=4)
val_loader = DataLoader(val_graphs, batch_size=4)

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## sGNN with GCN Encoder and 3 Features

In [269]:
def check_crisis_years(year_pairs, crisis_years):
    result = []
    for pair in year_pairs:
        start, end = pair
        # Check if any crisis year is between the pair or equals the later year
        if any(start < year <= end for year in crisis_years):
            result.append(0)
        else:
            result.append(1)
    return result

In [270]:
crisis_years = [1983, 1982, 2008, 2002, 2016, 1967, 1962, 1989, 2012, 1963, 1993, 1986, 1996,1978]

def get_year_pairs(year_range):
    return [(year1, year2) for year1 in year_range for year2 in year_range if year2 >= year1]

def get_loader_pairs(dataset):
    return [(dataset[i], dataset[j]) for i in range(len(dataset)) for j in range(len(dataset)) if j >= i]

def get_graph_pairs(graphs):
    return [(graphs[i], graphs[j]) for i in range(len(graphs)) for j in range(len(graphs)) if j >= i]

train_pairs = get_year_pairs(train_years)
val_pairs = get_year_pairs(val_years)

train_y = check_crisis_years(train_pairs, crisis_years)
val_y = check_crisis_years(val_pairs, crisis_years)

train_loader_pairs = get_loader_pairs(train_loader.dataset)
val_loader_pairs = get_loader_pairs(val_loader.dataset)

train_graph_pairs = get_graph_pairs(train_graphs)
val_graph_pairs = get_graph_pairs(val_graphs)

In [271]:
train_torch_y = torch.tensor(np.array(train_y))
val_torch_y = torch.tensor(np.array(val_y))

In [272]:
labeled_pairs_train = list(zip(train_loader_pairs, train_y))
labeled_pairs_val = list(zip(val_loader_pairs, val_y))

In [273]:
flattened_train = [(a, b, c) for ((a, b), c) in labeled_pairs_train]
flattened_val  = [(a, b, c) for ((a, b), c) in labeled_pairs_val]

In [191]:
import random
positive_samples = [item for item in flattened_train if item[2] == 1]
negative_samples = [item for item in flattened_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = data

# Shuffle the balanced dataset
random.shuffle(balanced_data)

In [252]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

model = SiameseGNN(num_features=balanced_data[0][0].num_node_features)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Adjust step_size and gamma as needed
criterion = nn.BCELoss()

for epoch in tqdm(range(20)):
    model.train()
    train_losses = []
    for data1, data2, label in balanced_data:

        optimizer.zero_grad()
        out = model(data1, data2)
        label = torch.tensor(label).view(1).float()
        loss = criterion(out.squeeze(0), label)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    scheduler.step()  # Add this line to update the learning rate

    model.eval()
    with torch.no_grad():
        val_losses = []
        correct = 0
        total = 0
        for data1, data2, label in flattened_val:
            out = model(data1, data2)
            label = torch.tensor(label).view(1).float()
            val_loss = criterion(out.squeeze(0), label)
            val_losses.append(val_loss.item())

            predictions = torch.round(out.squeeze())
            correct += (predictions == label).sum().item()
            total += 1

        val_loss = sum(val_losses) / len(val_losses)
        val_accuracy = correct / total

    print(f'Epoch: {epoch+1}, Training Loss: {sum(train_losses)/len(train_losses)}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:22<07:09, 22.63s/it]

Epoch: 1, Training Loss: 0.7069368174782505, Validation Loss: 0.6644339471152334, Validation Accuracy: 0.803030303030303


 10%|█         | 2/20 [00:44<06:35, 21.95s/it]

Epoch: 2, Training Loss: 0.7007183937708072, Validation Loss: 0.6638694134625521, Validation Accuracy: 0.7878787878787878


...

Epoch: 18, Training Loss: 0.41787379529199165, Validation Loss: 0.8951684424678343, Validation Accuracy: 0.6363636363636364


 95%|█████████▌| 19/20 [06:50<00:21, 21.88s/it]

Epoch: 19, Training Loss: 0.40456365760822904, Validation Loss: 0.91365475038765, Validation Accuracy: 0.6363636363636364


100%|██████████| 20/20 [07:12<00:00, 21.60s/it]

Epoch: 20, Training Loss: 0.39105719769102126, Validation Loss: 0.9060935187118241, Validation Accuracy: 0.6363636363636364


## sGNN with Feature Subset

In [ ]:
with open("feature_dicts/filtered_features_dict.pkl", "rb") as f:
    feat_dict = pkl.load(f)

In [ ]:
years = range(1962, 2019)

In [ ]:
from torch_geometric.data import DataLoader
test_loader = DataLoader(test_graphs, batch_size=32)
train_loader = DataLoader(train_graphs, batch_size=32)
val_loader = DataLoader(val_graphs, batch_size=32)

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
def add_features(years, graphs):

    zeros = torch.zeros(59)

    for i in range(len(years)):
        new_x = torch.empty(0, 59)
        year = years[i]

        feat_dict_year = feat_dict[year].combined_features

        for j, country in enumerate(all_nodes):
            if j == 0:
                new_x = torch.stack([zeros])

            elif country in feat_dict_year["country_code"].values:
                tensor_before = graphs[i].x[j]
                country_row = feat_dict_year[feat_dict_year["country_code"] == country]
                country_row = country_row.drop(columns = ["prev_gdp_growth", "country_code", "current_gdp_growth"])
                row_values = country_row.values.tolist()
                row_tensor = torch.tensor(row_values)[0]
                combined_values = torch.cat((tensor_before, row_tensor))

                new_x = torch.cat((new_x, combined_values.unsqueeze(0)), dim=0)

            else:
                new_x = torch.cat((new_x, zeros.unsqueeze(0)), dim=0)

        graphs[i].x = new_x

    return graphs

In [ ]:
train_graphs = add_features(train_years, train_graphs)
val_graphs = add_features(val_years, val_graphs)
test_graphs = add_features(test_years, test_graphs)

NameError: name 'train_years' is not defined

In [ ]:
crisis_years = [1983, 1982, 2008, 2002, 2016, 1967, 1962, 1989, 2012, 1963, 1993, 1986, 1996,1978]

def get_year_pairs(year_range):
    return [(year1, year2) for year1 in year_range for year2 in year_range if year2 >= year1]

def get_loader_pairs(dataset):
    return [(dataset[i], dataset[j]) for i in range(len(dataset)) for j in range(len(dataset)) if j >= i]

train_pairs = get_year_pairs(train_years)
val_pairs = get_year_pairs(val_years)

train_y = check_crisis_years(train_pairs, crisis_years)
val_y = check_crisis_years(val_pairs, crisis_years)

train_loader_pairs = get_loader_pairs(train_loader.dataset)
val_loader_pairs = get_loader_pairs(val_loader.dataset)

In [ ]:
train_torch_y = torch.tensor(np.array(train_y))
val_torch_y = torch.tensor(np.array(val_y))

In [ ]:
train_loader = train_loader_pairs
val_loader = val_loader_pairs

model = SiameseGNN(num_features=train_loader[0][0].num_node_features)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Adjust step_size and gamma as needed
criterion = nn.BCELoss()

for epoch in tqdm(range(10)):
    model.train()
    train_losses = []
    for i in range(len(train_loader)):

        optimizer.zero_grad()
        out = model(train_loader[i][0], train_loader[i][1])
        loss = criterion(out.squeeze(), train_torch_y[i].to(torch.float))
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    scheduler.step()  # Add this line to update the learning rate

    model.eval()
    with torch.no_grad():
        val_losses = []
        correct = 0
        total = 0
        for i in range(len(val_loader)):
            out = model(val_loader[i][0], val_loader[i][1])
            val_loss = criterion(out.squeeze(), val_torch_y[i].to(torch.float))
            val_losses.append(val_loss.item())

            predictions = torch.round(out.squeeze())
            correct += (predictions == val_torch_y[i]).sum().item()
            total += 1

        val_loss = sum(val_losses) / len(val_losses)
        val_accuracy = correct / total

    print(f'Epoch: {epoch+1}, Training Loss: {sum(train_losses)/len(train_losses)}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

 10%|█         | 1/10 [00:13<01:57, 13.00s/it]

Epoch: 1, Training Loss: 0.4229666137895664, Validation Loss: 0.4994966710607211, Validation Accuracy: 0.803030303030303


 20%|██        | 2/10 [00:25<01:42, 12.87s/it]

Epoch: 2, Training Loss: 0.41427947403628285, Validation Loss: 0.5009946116443836, Validation Accuracy: 0.803030303030303


 30%|███       | 3/10 [00:39<01:33, 13.30s/it]

...

Epoch: 8, Training Loss: 0.4073875575068117, Validation Loss: 0.5056198169336175, Validation Accuracy: 0.803030303030303


 90%|█████████ | 9/10 [04:19<00:22, 22.31s/it]

Epoch: 9, Training Loss: 0.4068174197393305, Validation Loss: 0.5069572094715002, Validation Accuracy: 0.803030303030303


100%|██████████| 10/10 [04:31<00:00, 27.13s/it]

Epoch: 10, Training Loss: 0.4066063172676984, Validation Loss: 0.508396189095396, Validation Accuracy: 0.803030303030303


## Random Feature Subset

In [ ]:
years = range(1962,2019)

train_years = [2005, 1969, 2002, 1997, 1993, 1982, 2001, 2000, 1962, 1985, 1978, 2016, 1986, 1987, 1989, 1971, 2013, 1996, 1995, 1967, 2017, 1974, 1990, 1977, 1980, 2014, 1965, 1984, 2006, 1973, 1968, 1981, 1970, 1991]
val_years = [1975, 1983, 2009, 1966, 1999, 1988, 2007, 1979, 1972, 2015, 2003]
test_years = [1963, 1964, 1976, 1992, 1994, 1998, 2004, 2008, 2010, 2011, 2012, 2018]

train_graphs = []
val_graphs = []
test_graphs = []
i = 0

for year in tqdm(years):
    print(str(year), end='\r')
    
    trade = TradeNetwork(year = year)
    trade.prepare_features()
    trade.prepare_network()
    trade.graph_create(node_features = ['prev_gdp_per_cap_growth', 'current_gdp_per_cap_growth',
    'resource_0', 'resource_1', 'resource_2', 'resource_3', 'resource_4', 'resource_5', 'resource_6', 'resource_7',
       'resource_8', 'resource_9'],
        node_labels = 'future_gdp_per_cap_growth')
    
    if(year in val_years):
        val_graphs.append(trade.pyg_graph)
    elif(year in test_years):
        test_graphs.append(trade.pyg_graph)
    else: 
        train_graphs.append(trade.pyg_graph)
        
    trade.features["year"] = year
    
    if(i == 0):
        trade_df = trade.features
    else: 
        trade_df = pd.concat([trade_df, trade.features])
        
    i = i+1
    print(trade.node_attributes.size())

  0%|          | 0/57 [00:00<?, ?it/s]

  2%|▏         | 1/57 [00:05<04:47,  5.13s/it]

torch.Size([199, 12])


  4%|▎         | 2/57 [00:10<04:47,  5.22s/it]

torch.Size([199, 12])


...

torch.Size([199, 12])


 98%|█████████▊| 56/57 [10:31<00:19, 19.09s/it]

torch.Size([199, 12])


100%|██████████| 57/57 [10:51<00:00, 11.43s/it]

torch.Size([199, 12])


In [ ]:
import pickle as pkl
with open("pygcn/train_graphs.pickle", "wb") as f:
    pkl.dump(train_graphs, f)

with open("pygcn/val_graphs.pickle", "wb") as f:
    pkl.dump(val_graphs, f)

with open("pygcn/test_graphs.pickle", "wb") as f:
    pkl.dump(test_graphs, f)

In [ ]:
with open("feature_dicts/random_features_dict.pkl", "rb") as f:
    feat_dict_random = pkl.load(f)

In [ ]:
def add_features(years, graphs, feat_dict):

    zeros = torch.zeros(434)

    for i in range(len(years)):
        new_x = torch.empty(0, 434)
        year = years[i]

        feat_dict_year = feat_dict[year].combined_features

        for j, country in enumerate(all_nodes):
            if j == 0:
                new_x = torch.stack([zeros])

            elif country in feat_dict_year["country_code"].values:
                tensor_before = graphs[i].x[j]
                country_row = feat_dict_year[feat_dict_year["country_code"] == country]
                country_row = country_row.drop(columns = ["prev_gdp_growth", "country_code", "current_gdp_growth"])
                row_values = country_row.values.tolist()
                row_tensor = torch.tensor(row_values)[0]
                combined_values = torch.cat((tensor_before, row_tensor))

                new_x = torch.cat((new_x, combined_values.unsqueeze(0)), dim=0)

            else:
                new_x = torch.cat((new_x, zeros.unsqueeze(0)), dim=0)

        graphs[i].x = new_x

    return graphs

In [ ]:
train_graphs = add_features(train_years, train_graphs, feat_dict_random)
val_graphs = add_features(val_years, val_graphs, feat_dict_random)
test_graphs = add_features(test_years, test_graphs, feat_dict_random)

In [ ]:
from torch_geometric.data import DataLoader
test_loader = DataLoader(test_graphs, batch_size=32)
train_loader = DataLoader(train_graphs, batch_size=32)
val_loader = DataLoader(val_graphs, batch_size=32)

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
def check_crisis_years(year_pairs, crisis_years):
    result = []
    for pair in year_pairs:
        start, end = pair
        # Check if any crisis year is between the pair or equals the later year
        if any(start < year <= end for year in crisis_years):
            result.append(0)
        else:
            result.append(1)
    return result

In [ ]:
crisis_years = [1983, 1982, 2008, 2002, 2016, 1967, 1962, 1989, 2012, 1963, 1993, 1986, 1996,1978]

def get_year_pairs(year_range):
    return [(year1, year2) for year1 in year_range for year2 in year_range if year2 >= year1]

def get_loader_pairs(dataset):
    return [(dataset[i], dataset[j]) for i in range(len(dataset)) for j in range(len(dataset)) if j >= i]

train_pairs = get_year_pairs(train_years)
val_pairs = get_year_pairs(val_years)

train_y = check_crisis_years(train_pairs, crisis_years)
val_y = check_crisis_years(val_pairs, crisis_years)

train_loader_pairs = get_loader_pairs(train_loader.dataset)
val_loader_pairs = get_loader_pairs(val_loader.dataset)

In [ ]:
train_torch_y = torch.tensor(np.array(train_y))
val_torch_y = torch.tensor(np.array(val_y))

In [ ]:
N_neg = torch.eq(train_torch_y, 0).sum().item()
N_pos = torch.eq(train_torch_y, 1).sum().item()

# Calculate the weight for the positive class
weight_for_positive_class = N_neg / N_pos
pos_weight = torch.tensor([weight_for_positive_class])